In [6]:
# Basic Python Libs - Not everything will be used here...
import os
import sys
import importlib
import csv
import pydotplus
import plotly
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.cm as cm
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, learning_curve
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, auc, roc_auc_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression
import joblib
np.set_printoptions(threshold=sys.maxsize)

# Own Imports
from Parameters import *
from Classifier_Trainer import *
from Classifier_Validation_Plots import *

Now I'm going to train a model. Before I can do that I need to label and encode the categorial variables (Since categorial features are not supported in sklearn...)

In [7]:
# Paths, functions and other useful stuff
data_path = "..\\Data\\"
file_name = "challenge_data.csv"

classifier_path = classifier_name[0]
if not os.path.exists(classifier_path):
    os.makedirs(classifier_path)
model_path = classifier_path+"\Model\\"
if not os.path.exists(model_path):
    os.makedirs(model_path)
plot_path = classifier_path+"\Plots"
if not os.path.exists(plot_path):
    os.makedirs(plot_path)
tree_path = plot_path+"\Decision_Trees\\"
if not os.path.exists(tree_path):
    os.makedirs(tree_path)
correlation_path = plot_path+"\Correlations"
if not os.path.exists(correlation_path):
    os.makedirs(correlation_path)
training_path = plot_path+"\Training\\"
if not os.path.exists(training_path):
    os.makedirs(training_path)
validation_path = plot_path+"\Validation\\"
if not os.path.exists(validation_path):
    os.makedirs(validation_path)
visualization_path = plot_path+"\Visualization\\"
if not os.path.exists(visualization_path):
    os.makedirs(visualization_path)

In [8]:
# Data import and dummy variables
df = pd.read_csv(data_path+file_name, delimiter=',')
df = df.drop(columns=['Unnamed: 0'])

cat_cols = ['category', 'product_group', 'color', 'brand']
df_dummies = pd.get_dummies(df, prefix_sep="_",
                              columns=cat_cols)

size_values = ['S', 'M', 'L', 'XL', 'XXL']
size_label = ['0', '1', '2', '3', '4']
df_dummies = df_dummies.replace(to_replace=size_values, value=size_label)
print(df_dummies.columns)

Index(['age', 'height_in_cm', 'date_shipped', 'size', 'itret',
       'shipping_country', 'weight_in_kg', 'price', 'category_Belts',
       'category_Blazers',
       ...
       'brand_Brand76', 'brand_Brand77', 'brand_Brand78', 'brand_Brand79',
       'brand_Brand8', 'brand_Brand80', 'brand_Brand81', 'brand_Brand82',
       'brand_Brand83', 'brand_Brand9'],
      dtype='object', length=171)


Now we have our dummy variables and we labled our size feature accordingly.
In order to train our model, we will need to split our dataset into at least one training and one testing sample. I would invest more time in a better splitting, but given the timeframe I'll settle with a very simple splitting.
Next up we define our features.

In [9]:
#Feature_List = df_dummies.columns.tolist()
#Feature_List.remove('itret')
#Feature_List.remove('date_shipped')
#print(Feature_List)

X = df_dummies[Feature_List] # Indepent variables
y = df_dummies['itret'] # Dependent Variables

# Split
X_dev, X_eval, y_dev, y_eval = train_test_split(X, y, test_size=reserve_fraction, random_state=42)
print("\nReserving " + str(format(reserve_fraction*100, '.2f')) + "% Of Events For Later Use: " + str(format(len(X_eval), ',.0f')))
print("\nUsing " + str(format((1-reserve_fraction)*100, '.2f')) + "% Of Events For Splitting: " + str(format(len(X_dev), ',.0f')))
X_train, X_test, y_train, y_test = train_test_split(X_dev, y_dev, test_size=test_fraction, random_state=random_seeds[0])
print("\nUsing " + str(format((1-reserve_fraction)*(1-test_fraction)*100, '.2f')) + "% Of Events For Training: " + str(format(len(X_train), ',.0f')))
print("\nUsing " + str(format((1-reserve_fraction)*test_fraction*100, '.2f')) + "% Of Events For Testing: " + str(format(len(X_test), ',.0f')))
print("\nData Has Been Split...")


Reserving 5.00% Of Events For Later Use: 7,344

Using 95.00% Of Events For Splitting: 139,522

Using 47.50% Of Events For Training: 69,761

Using 47.50% Of Events For Testing: 69,761

Data Has Been Split...


In [10]:
# parameters = {
#     "loss":["deviance"],
#     "min_samples_leaf": np.linspace(0.05, 0.2, 4),
#     "max_depth": range(2,6),
#     "criterion": ["friedman_mse"],
#     "subsample":[0.8, 0.9, 1.0],
#     "learning_rate": [0.01, 0.05, 0.1],
#     "n_estimators": np.arange(50, 300, 50)
#     }


# # run randomized search
# print("Defining the Gridsearch")
# clf = GridSearchCV(GradientBoostingClassifier(), parameters, scoring='roc_auc', cv=3, n_jobs=4, verbose=2)
# print("Training on all gridpoints...")
# clf.fit(X_train, y_train)
# print('Gradient boosting trees best params:', clf.best_params_)
# print('Gradient boosting trees score:', clf.best_score_)

Define our model, train it and evaluate.

In [14]:
# GB_clfs = {}
# GB_clfs[str(classifier_name[0])] = GradientBoostingClassifier(max_depth=max_depth, 
#                                                               min_samples_leaf=min_samples_leaf, 
#                                                               random_state=0, subsample=subsample, 
#                                                               n_estimators=n_estimators, 
#                                                               learning_rate=learning_rate, 
#                                                               verbose=True)
# clfs=[]
# print("\nCrosstraining - 0")
# classifier_training(X_train, y_train, X_test, y_test, clfs, 0, 
#                     GB_clfs[classifier_name[0]], model_path, tree_path, training_path, False)
# print("---"*42)
# print("Crosstraining - 1")
# classifier_training(X_train, y_train, X_test, y_test,clfs,  1, 
#                     GB_clfs[classifier_name[0]], model_path, tree_path, training_path, False)
# print("\nFinished Training...")
# print("---"*42)
# clfs[0].set_params(verbose=False)
# clfs[1].set_params(verbose=False)

logisticRegr = LogisticRegression(solver = 'liblinear', max_iter = 5000)
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, target_names=["background", "signal"]))

[[24391 10472]
 [11507 23391]]
              precision    recall  f1-score   support

  background       0.68      0.70      0.69     34863
      signal       0.69      0.67      0.68     34898

    accuracy                           0.68     69761
   macro avg       0.69      0.68      0.68     69761
weighted avg       0.69      0.68      0.68     69761



In [12]:
Ntree_ROC_Curve_Figname = validation_path + str(classifier_name[0]) + "_Tree_" + str(len(clfs[0].estimators_)) + "_ROC.png"
plot_Ntree_ROC_curve(clfs[0], (X_train,y_train),(X_test,y_test), Ntree_ROC_Curve_Figname)
Ntree_PR_Curve_Figname = validation_path + str(classifier_name[0]) + "_Tree_" + str(len(clfs[0].estimators_)) + "_PR.png"
plot_Ntree_PR_curve(clfs[0], (X_train,y_train),(X_test,y_test), Ntree_PR_Curve_Figname)

# Reference For Some Of These Numbers
# P - condition positive, the number of real positive cases in the data
# N - condition negative, the number of real negative cases in the data

# TP - true positive, eqv. with hit

# TN - true negative,  eqv. with correct rejection
# FP - false positive, eqv. with false alarm, Type I error
# FN - false negative,  eqv. with miss, Type II error

# PPV - Precision = TP / (TP + FP) = 1 - FDR, positive predictive value
# TPR - Recall = TP / (TP + FN) = 1 - FNR, true positive rate
# FNR - Miss Rate = FN / (FN + TP) = 1 - TPR, false negative rate
# FDR - False Discovery Rate = FP / (FP + TP)
# ACC - Accuracy = (TP + TN) / (TP + TN + FP + FN)
# F betta - F betta Score = (1+ betta**2) * ( PPV * TPR) / (betta**2 * PPV + TPR), is the harmonic mean of precision and recall

# If I ever want to plot multiple clfs
#fig, axes = plt.subplots(nrows=len(clfs), sharex=True)
#for clf, ax in zip(clfs, axes):  
#	plot_learning_curve(clf, "Learning curves", X_dev, y_dev, scoring='roc_auc', n_jobs=7, cv=4, ax=ax, xlabel=False)
#axes[0].legend(loc="best")
#axes[-1].set_xlabel("Training examples")

print("Plotting Performance vs. Size Of Training Set...")
fig, axis = plt.subplots(nrows=1, sharex=True)
plot_learning_curve(clfs[0], "Learning curves", X_dev, y_dev, n_jobs=n_jobs, cv=cv, ax=axis)
axis.legend(loc="best")
axis.set_xlabel("Training examples")
fig_name = validation_path + str(classifier_name[0]) + "_Score_Validation" + ".png"
plt.savefig(fig_name)
plt.close()
print("\nFinished Validation...")
print("---"*42)

NameError: name 'clfs' is not defined